# Multiple Outputs (DDF)

 - Author : Sheng Dai (sheng.dai@aalto.fi)
 - Date : June 16, 2020

References:

[1] Kuosmanen, T. and Johnson, A. (2017). Modeling joint production of multiple outputs in StoNED: Directional distance function approach, European Journal of Operational Research. 262, pp. 792–801.

## Example

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import CNLSDDF

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [4]:
# outputs
y1 = df['Energy']
y1 = np.asmatrix(y1).T
y2 = df['Length']
y2 = np.asmatrix(y2).T
y3 = df['Customers']
y3 = np.asmatrix(y3).T
y = np.concatenate((y1, y2, y3), axis=1)

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [5]:
# define and solve the CNLS model

gx  = [1.0, 0.25]
gy  = [0.0, 0.0, 0.0]

cet = "addi"
fun = "prod"

model = CNLSDDF.cnlsddf(y, x, fun, gx, gy)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 8010            
  Cones                  : 0               
  Scalar variables       : 623             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.02    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 89
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0              

In [8]:
# display the estimates (alpha, beta, gamma, and residual)
model.a.display()
model.b.display()
model.g.display()
model.e.display()

a : alpha
    Size=89, Index=i
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :  None : -191.29017469078738 :  None : False : False :  Reals
      1 :  None :  -234.6007463392647 :  None : False : False :  Reals
      2 :  None : -293.22604913384265 :  None : False : False :  Reals
      3 :  None : -145.44382702414055 :  None : False : False :  Reals
      4 :  None :  -221.4867949156726 :  None : False : False :  Reals
      5 :  None : -322.28441096944283 :  None : False : False :  Reals
      6 :  None :  -239.4838877462972 :  None : False : False :  Reals
      7 :  None :  -191.2216849120775 :  None : False : False :  Reals
      8 :  None :  -302.3138216417427 :  None : False : False :  Reals
      9 :  None : -182.99596903982243 :  None : False : False :  Reals
     10 :  None :  3464.7362182006377 :  None : False : False :  Reals
     11 :  None :  2886.4143459802085 :  None : False : False :  Reals
     12 :  None :  -302.3202362426989 :  None 

In [9]:
# retrive the alpha
val = list(model.a[:].value)
alpha = np.asarray(val)
alpha

array([-191.29017469, -234.60074634, -293.22604913, -145.44382702,
       -221.48679492, -322.28441097, -239.48388775, -191.22168491,
       -302.31382164, -182.99596904, 3464.7362182 , 2886.41434598,
       -302.32023624, -260.55141148, 3106.07908985, -150.763414  ,
       -302.44921124, -260.58090353,  149.67056486, -132.16009633,
       -166.3431538 , -255.49536012, -407.71132805,  102.43607718,
       -239.49466898, -260.26594089, -187.06512567, 2601.33930557,
       -295.07582597, -327.84664   , -260.63103908, 2119.10622988,
       -208.16703853, -354.04239246, -196.95574093, -239.47269241,
       -187.79630389, -247.23444385, -272.81577215,  -73.45194147,
       -145.27276018, -285.12903439,  935.62978676, -258.80761728,
       -279.87851967, -189.64262016, -149.94422652, -355.01730019,
        701.43412156,  806.97416809, -252.42885529, -184.97865334,
       -302.31927402, -163.18816254, -260.71084015, 1931.12728369,
        -17.64998316, -245.69286872, -188.29591645,  195.63854

In [10]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

array([-2.95746956e+01, -1.26611634e+02,  1.40185127e+02,  1.39917104e+03,
       -4.21351524e+01, -2.41780497e+02, -5.80911160e-01,  1.48335204e+02,
        1.89063486e+02, -4.51363316e+01,  3.03452411e+02, -5.19258574e+02,
        2.76126378e+02,  5.42088096e+02, -3.15707239e+01, -9.10852350e+01,
       -2.50334939e+01,  1.10951392e+02, -1.06915589e+02, -1.87141235e+02,
       -2.93822441e+02, -3.89905927e+02, -2.58723219e+02, -1.71057729e+02,
       -1.38679701e+02,  8.38391837e+01,  2.19588979e+02, -8.53233413e+02,
       -1.94562600e+02,  1.78228349e+02, -1.41982838e+00, -4.70376771e+02,
       -1.81525643e+01,  2.82447501e+02, -1.77147938e+01,  9.43264804e+02,
       -1.32365587e+02, -1.37725932e+02, -1.41439386e+02,  2.27800615e+02,
        5.49971107e-03, -5.10169898e+01,  3.04601672e+02, -6.42465955e+01,
       -7.15859494e+01, -1.65499548e+02,  3.63360381e+02, -1.04109068e+02,
       -1.69738162e+02, -1.09615657e+02, -9.54136554e+01,  1.91969553e+02,
        5.24888824e+01,  

In [11]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:, :].value)
beta = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

beta = pd.DataFrame(beta, columns=['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['OPEX', 'CAPEX']
beta

,OPEX,CAPEX
Name,,
0.0,0.985177,5.929064e-02
1.0,0.855150,5.794014e-01
2.0,0.999999,4.808348e-06
3.0,1.000000,1.085417e-07
4.0,0.991687,3.325121e-02
...,...,...
84.0,0.885218,4.591285e-01
85.0,0.349658,2.601369e+00
86.0,0.932790,2.688398e-01


In [12]:
# retrive the gamma
ind = list(model.g)
val = list(model.g[:, :].value)
gamma = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

gamma = pd.DataFrame(gamma, columns=['Name', 'Key', 'Value'])
gamma = gamma.pivot(index='Name', columns='Key', values='Value')
gamma.columns = ['Energy', 'Length', 'Customers']
gamma

,Energy,Length,Customers
Name,,,
0.0,4.130744,0.267803,1.515532e-03
1.0,0.119805,0.380118,6.279310e-02
2.0,3.328429,0.055598,1.728273e-02
3.0,4.240829,0.247889,2.504584e-04
4.0,0.168888,0.363045,1.322300e-03
...,...,...,...
84.0,4.825101,0.429539,7.555826e-08
85.0,0.742610,0.728224,1.026605e-01
86.0,3.433396,0.172359,2.047893e-02
